In [48]:
import numpy as np
from sklearn.neighbors import radius_neighbors_graph, KDTree, kneighbors_graph
import chainer
import chainer.functions as F
import glob
import struct
import cupy
from chainer import cuda
import time
import utils

In [2]:
'''
Data
'''
num_particles = 32
zX = 0.6
X = utils.load_datum(num_particles, zX, normalize_data=True)
N = X.shape[0]

In [3]:
X_coo = X[...,:3]

In [ ]:
def csr_select_mean(csr, x):
    y = np.copy(x)
    N = x.shape[0]
    for i in range(N):
        j, k = csr.indptr[i], csr.indptr[i+1]
        cur_ind = csr.indices[j:k]
        num_ind = cur_ind.shape[0]
        y[i] = np.mean(y[cur_ind], axis=0)
    return y

def batch_csr_select_mean(csr, x):
    mb_size, N, D = x.shape
    xflat = np.reshape(x, (-1, D))
    y = np.copy(xflat)
    # csr is flattend dims
    for i in range(xflat.shape[0]):
        a,b = a,b = csr.indptr[i], csr.indptr[i+1]
        cur_ind = csr.indices[a:b]
        y[i] = np.mean(xflat[cur_ind], axis=0)
    return np.reshape(y, (x.shape))

def matmul_select_mean(arr, x):
    summed = np.sum(arr, axis=-1)
    graph_div = graph_arr / summed
    return np.matmul(graph_div, x)

def batch_matmul_select_mean(arr, x):
    mb_size, N, D = x.shape
    xflat = np.reshape(x, (-1, D))
    summed = np.sum(arr, axis=-1)
    #graph_div = graph_arr / summed
    y = np.matmul(arr / summed, xflat)
    return np.reshape(y, (mb_size, N, D))

In [4]:
rad = 0.035
x = X_coo[13]
graph = radius_neighbors_graph(x, rad, include_self=True)
#coograph = graph.tocoo()
indptr  = graph.indptr
data    = graph.data
indices = graph.indices

In [53]:
indices.shape

(850502,)

In [55]:
indptr.shape

(32769,)

In [49]:
graph_knn = kneighbors_graph(x,14,include_self=True).indices

In [50]:
graph_knn.shape

(458752,)

In [51]:
graph.indices.shape

(850502,)

In [28]:
y = x[indices]

In [54]:
y.shape

(850502, 3)

In [31]:
z = y[indptr[:-1]]

In [39]:
indptr[1]

4

In [33]:
z

array([[0.04546474, 0.04252362, 0.04149579],
       [0.08151047, 0.04890734, 0.05004762],
       [0.08151047, 0.04890734, 0.05004762],
       ...,
       [0.8786178 , 0.01884086, 0.9687873 ],
       [0.934382  , 0.00308838, 0.99381256],
       [0.97543377, 0.99976265, 0.9921783 ]], dtype=float32)

In [44]:
for i in range(x.shape[0]):
    cur_rows = lil_graph.rows[i]
    if np.all(z[i] == x[i]):
        print('{}: {}'.format(i, cur_rows))
    #if np.all(z[i] != x[i]):print('{}: {}'.format(i, lil_graph.rows[i]))

0: [0, 993, 1024, 2016]
1: [1, 2, 33, 993, 994, 1025, 2017, 2018, 2019, 3041, 3042]
5: [5]
6: [6, 7, 8]
10: [10, 11, 1002, 3048, 4072, 4073]
12: [12, 1036, 2059, 3049]
13: [13]
14: [14]
15: [15, 16, 1007, 1008]
17: [17, 49, 1009, 31762]
18: [18, 19, 50, 51, 1010, 1011, 1042, 31762, 31763, 31795]
20: [20, 21, 52, 85, 1012, 1076, 31764]
22: [22, 54, 55, 86, 1013, 1046, 1078]
23: [23, 24, 55, 56]
25: [25]
26: [26, 27, 58]
30: [30, 31, 61, 1054, 31774]
32: [32, 65, 1024, 2111]
35: [35, 36, 66, 1028, 1059, 1060, 1091, 1122, 2052, 2083, 2084, 2115, 2146, 3108, 3139]
37: [37, 68, 99, 100, 131, 165, 1061, 1062, 31812, 31843, 31844]
38: [38, 39, 40, 41, 70, 73, 74, 105, 166, 30789, 30820, 30822, 31781, 31815, 31816, 31817, 31818, 31912]
42: [42, 43, 1031, 1032, 1065, 2056, 2089, 2090]
44: [44]
45: [45]
46: [46, 47]
53: [53, 1012, 1044, 1045, 1076, 1077, 2036]
57: [57]
64: [64]
69: [69, 101, 102, 132, 133, 134, 135, 136, 137, 164, 165, 167, 168, 198, 1094, 1095, 1096, 1097, 1125, 1126, 1127, 112

24489: [24489]
24498: [24498, 24499, 25490, 25521, 25522, 25523, 25588, 25620, 26515, 26545, 26546, 26547, 26612, 26644, 27570, 27604, 27636]
24500: [24500, 24567, 24597, 24599, 25492, 25525, 25526, 25557, 25558, 25590, 25591, 25592, 25621, 26516, 26517, 26548, 26550, 26581, 26582, 26583, 26613, 26615, 27605, 27637]
24506: [24506, 24538, 24570]
24520: [24520]
24521: [24521, 24522]
24547: [24547]
24549: [24549, 24550]
24553: [24553]
24554: [24554]
24555: [24555]
24576: [24576]
24577: [24577]
24578: [24578]
24579: [24579]
24580: [24580]
24584: [24584, 24585]
24602: [24602]
24603: [24603, 24604]
24605: [24605]
24606: [24606]
24607: [24607]
24608: [24608, 24609, 24640]
24610: [24610, 24642]
24611: [24611, 24612, 24643]
24630: [24630, 25624, 26646, 26647, 26678]
24633: [24633]
24634: [24634]
24635: [24635, 24636, 25659]
24637: [24637, 24638, 24669]
24665: [24665]
24666: [24666]
24667: [24667, 24668, 25691, 25692]
24671: [24671]
24672: [24672]
24673: [24673]
24674: [24674]
24675: [24675]
246

In [24]:
import cupy
cupy_csr = cupy.sparse.csr_matrix(graph)

In [26]:
cupy_csr.shape

(32768, 32768)

In [9]:
lil_graph = graph.tolil()

In [65]:
lil_graph.data = np.random.rand(*lil_graph.data.shape)

In [66]:
lil_graph.data

array([0.65431973, 0.72087312, 0.12908765, ..., 0.84799103, 0.90403647,
       0.21765206])

In [7]:
dense_graph = graph.toarray()

In [18]:
dense_diffed = dense_graph / np.diff(indptr)

In [46]:
dense_diffed.shape

(32768, 32768)

In [20]:
Y = np.matmul(dense_diffed, x)

In [21]:
Y.shape

(32768, 3)

In [71]:
# for 113th nn
'''
F.select_item(x,t)
    x: (N, D)
    t: (N,), where elems in t index into D
    y[i] = x[i, t[i]] for all i

def csr_select_mean(csr, x):
    y = np.copy(x)
    N = x.shape[0]
    for i in range(N):
        j, k = csr.indptr[i], csr.indptr[i+1]
        cur_ind = csr.indices[j:k]
        num_ind = cur_ind.shape[0]
        y[i] = np.mean(y[cur_ind], axis=0)
    return y
'''
inds_113 = indices[indptr[113]:indptr[114]]

In [73]:
X_coo[13][inds_113].shape

(2, 3)

In [41]:
y = np.copy(X_coo[13])

In [49]:
inds = indices[indptr[:-1]]

In [56]:
for i in range(N):
    cur = np.copy(X_coo[i])
    graph = radius_neighbors_graph(cur,rad,include_self=True)
    inds = graph.indices
    print('X[csr_indices].shape: {}'.format(cur[inds].shape))

X[csr_indices].shape: (651986, 3)
X[csr_indices].shape: (686652, 3)
X[csr_indices].shape: (684676, 3)
X[csr_indices].shape: (714096, 3)
X[csr_indices].shape: (680318, 3)
X[csr_indices].shape: (833820, 3)
X[csr_indices].shape: (952976, 3)
X[csr_indices].shape: (811310, 3)
X[csr_indices].shape: (732968, 3)
X[csr_indices].shape: (727308, 3)
X[csr_indices].shape: (772232, 3)
X[csr_indices].shape: (1553686, 3)
X[csr_indices].shape: (1054742, 3)
X[csr_indices].shape: (850502, 3)
X[csr_indices].shape: (1234776, 3)
X[csr_indices].shape: (703530, 3)
X[csr_indices].shape: (1171374, 3)
X[csr_indices].shape: (811582, 3)
X[csr_indices].shape: (782468, 3)
X[csr_indices].shape: (868794, 3)
X[csr_indices].shape: (819348, 3)
X[csr_indices].shape: (856292, 3)
X[csr_indices].shape: (1348554, 3)
X[csr_indices].shape: (819340, 3)
X[csr_indices].shape: (847782, 3)
X[csr_indices].shape: (775898, 3)
X[csr_indices].shape: (909292, 3)
X[csr_indices].shape: (921854, 3)
X[csr_indices].shape: (824506, 3)
X[csr_ind

KeyboardInterrupt: 

In [58]:
x13 = X_coo[13]
AS = radius_neighbors_graph(x13, rad, include_self=True)
poo = np.add.reduceat(x13[AS.indices], AS.indptr[:-1], axis=0) / np.diff(AS.indptr)[:, None]

In [67]:
AS.indptr

array([     0,      4,     15, ..., 850500, 850501, 850502], dtype=int32)

In [66]:
np.diff(AS.indptr)

array([ 4, 11, 20, ...,  3,  1,  1], dtype=int32)

In [63]:
AS.indptr[:-1].shape

(32768,)

In [62]:
x13[AS.indices].shape

(850502, 3)

In [32]:
nonzeros = np.zeros((X.shape[-2]))
for i in range(X.shape[-2]):
    nonzeros[i] = np.count_nonzero(grapharr[i])
    #print('{}'.format(np.count_nonzero(grapharr[i])))

In [33]:
np.median(nonzeros)

10.0

In [ ]:
# kdtree query time
# actually, if you ahve to get make a kdtree for all n in N, then just use radius_neighbors_graph??
tree_time_start = time.time()
kdtree = KDTree(X)
indices = {}
search_times = np.zeros((N))
for i in range(N):
    cur_time = time.time()
    cur_tree = KDTree(X[i])
    
    indices.append(kdtree.query_radius(X[i]))
    indices.append()

In [ ]:
indices = {}
search_times = np.zeros((N))
for i in range(N):
    cur_time = time.time()
    cur_tree = KDTree(X[i])
    
    indices.append(kdtree.query_radius(X[i]))
    indices.append()

In [6]:
ind, dist = kdtree.query_radius(X[0], r=0.3)

for i in range(N):
    
print(tree.query_radius(Y[[0]], r=0.3, count_only=True))
ind, dist = tree.query_radius(Y[[0]], r=0.3, return_distance=True)

(1000, 4096, 6)

In [5]:
'''
Density stuff
LOOK AT sklearn.neighbors module,
 - KDTree
 - Ball tree
 - radius neighbors
'''
rad = 0.01
x = X[0,:,:3]

In [6]:
graph = radius_neighbors_graph(x[:,:3], rad, include_self=True)
kdtree = KDTree(x)
coograph = graph.tocoo()
indptr  = graph.indptr
data    = graph.data
indices = graph.indices
print('graph.shape{} data.shape: {}, indices.shape: {}, indptr.shape: {}'.format(graph.shape, data.shape, indices.shape, indptr.shape))

graph.shape(4096, 4096) data.shape: (4876,), indices.shape: (4876,), indptr.shape: (4097,)


In [106]:
print(kdtree.query_radius(x, r=0.01))

[array([0]) array([1]) array([2]) ..., array([4093]) array([4094])
 array([4095])]


In [156]:
def make_kdtree(x, leaf_size):
    xflat = np.reshape(x, (-1, x.shape[-1]))
    t0 = time.time()
    kdtree = KDTree(xflat, leaf_size=leaf_size)
    print('{} time: {}'.format(leaf_size, time.time() - t0))
    return kdtree

x8 = X[:8]
kdtree40 = make_kdtree(x8, leaf_size=4096)

4096 time: 0.0036771297454833984


In [ ]:
class DensityNN():
    def __init__(self, X_in, rad, leaf_size=40):
        self.rad = rad
        mb_size, N, D = X_in.shape
        self.batch_idx = [i*N for i in range(mb_size)]
        self.kdtree = self.create_kdtree(X_in, leaf_size)
    
    def create_kdtree(self, X_in, leaf_size):
        r = self.rad
        mb_size, N, D = X_in.shape
        x = np.reshape(X_in, (-1, D))
        kdtree = KDTree(x, leaf_size=leaf_size)
        
        

In [157]:
x.shape

(4096, 3)

In [162]:
x1605n = x[kdtree.query_radius(x[[1605]], r=0.01).item()]
kdtree.query_radius(x, r=0.01)

array([array([0]), array([1]), array([2]), ..., array([4093]),
       array([4094]), array([4095])], dtype=object)

In [163]:
a = np.random.rand(4000, 3)

In [170]:
radius_neighbors_graph(a, 0.01, include_self=True).nnz

4078

In [114]:
np.random.seed(0)
Y = np.random.random((10,3))
tree = KDTree(Y, leaf_size=2)
print(tree.query_radius(Y[[0]], r=0.3, count_only=True))
ind, dist = tree.query_radius(Y[[0]], r=0.3, return_distance=True)
print(ind)
print(dist)

[3]
[array([3, 0, 1])]
[array([ 0.19662693,  0.        ,  0.29473397])]


In [88]:
print('ROW: IND, DATA')
for i in range(x.shape[0]):
    j,k = indptr[i], indptr[i+1]
    cur_ind = indices[j:k]
    cur_data = data[j:k]
    print('{}: {}, {}'.format(i, cur_ind, cur_data))
    

ROW: IND, DATA
0: [0], [ 1.]
1: [1], [ 1.]
2: [2], [ 1.]
3: [3], [ 1.]
4: [4], [ 1.]
5: [261 277   5], [ 1.  1.  1.]
6: [6], [ 1.]
7: [7], [ 1.]
8: [ 8 24], [ 1.  1.]
9: [9], [ 1.]
10: [10], [ 1.]
11: [11], [ 1.]
12: [12 13], [ 1.  1.]
13: [12 13], [ 1.  1.]
14: [14], [ 1.]
15: [15], [ 1.]
16: [16], [ 1.]
17: [17], [ 1.]
18: [18], [ 1.]
19: [19], [ 1.]
20: [ 20 517], [ 1.  1.]
21: [ 21 278], [ 1.  1.]
22: [262  22], [ 1.  1.]
23: [23], [ 1.]
24: [ 8 24], [ 1.  1.]
25: [25], [ 1.]
26: [26], [ 1.]
27: [27], [ 1.]
28: [29 28], [ 1.  1.]
29: [29 28], [ 1.  1.]
30: [30], [ 1.]
31: [31], [ 1.]
32: [32], [ 1.]
33: [33], [ 1.]
34: [34], [ 1.]
35: [35], [ 1.]
36: [36], [ 1.]
37: [37], [ 1.]
38: [38], [ 1.]
39: [39], [ 1.]
40: [40], [ 1.]
41: [41], [ 1.]
42: [42], [ 1.]
43: [43], [ 1.]
44: [44], [ 1.]
45: [45], [ 1.]
46: [46], [ 1.]
47: [47], [ 1.]
48: [48], [ 1.]
49: [49], [ 1.]
50: [50], [ 1.]
51: [51], [ 1.]
52: [52], [ 1.]
53: [53], [ 1.]
54: [70 54], [ 1.  1.]
55: [55], [ 1.]
56: [56], [ 1.

1297: [1297], [ 1.]
1298: [1298], [ 1.]
1299: [1299], [ 1.]
1300: [1300], [ 1.]
1301: [1301], [ 1.]
1302: [1302], [ 1.]
1303: [1303], [ 1.]
1304: [1304], [ 1.]
1305: [1305], [ 1.]
1306: [1306], [ 1.]
1307: [1307], [ 1.]
1308: [1564 1308], [ 1.  1.]
1309: [1309], [ 1.]
1310: [1310], [ 1.]
1311: [1311], [ 1.]
1312: [1312], [ 1.]
1313: [1313], [ 1.]
1314: [1314], [ 1.]
1315: [1315], [ 1.]
1316: [1316 1556], [ 1.  1.]
1317: [1317 1573], [ 1.  1.]
1318: [1318], [ 1.]
1319: [1319], [ 1.]
1320: [1320], [ 1.]
1321: [1321], [ 1.]
1322: [1322], [ 1.]
1323: [1323], [ 1.]
1324: [1324], [ 1.]
1325: [1325], [ 1.]
1326: [1326], [ 1.]
1327: [1327], [ 1.]
1328: [1328], [ 1.]
1329: [1329], [ 1.]
1330: [1330], [ 1.]
1331: [1331], [ 1.]
1332: [1588 1332], [ 1.  1.]
1333: [1333], [ 1.]
1334: [1334], [ 1.]
1335: [1335], [ 1.]
1336: [1336], [ 1.]
1337: [1337], [ 1.]
1338: [1339 1338], [ 1.  1.]
1339: [1339 1356 1338 1099], [ 1.  1.  1.  1.]
1340: [1340], [ 1.]
1341: [1341], [ 1.]
1342: [1342], [ 1.]
1343: [1

2639: [2639], [ 1.]
2640: [2640], [ 1.]
2641: [2641], [ 1.]
2642: [2642], [ 1.]
2643: [2643], [ 1.]
2644: [2628 2644], [ 1.  1.]
2645: [2645], [ 1.]
2646: [2646], [ 1.]
2647: [2647], [ 1.]
2648: [2648], [ 1.]
2649: [2649], [ 1.]
2650: [2650], [ 1.]
2651: [2651], [ 1.]
2652: [2652], [ 1.]
2653: [2653], [ 1.]
2654: [2654], [ 1.]
2655: [2655], [ 1.]
2656: [2656], [ 1.]
2657: [2657], [ 1.]
2658: [2658], [ 1.]
2659: [2659], [ 1.]
2660: [2660], [ 1.]
2661: [2661 2390], [ 1.  1.]
2662: [2662], [ 1.]
2663: [2663], [ 1.]
2664: [2664], [ 1.]
2665: [2665], [ 1.]
2666: [2666], [ 1.]
2667: [2667], [ 1.]
2668: [2668], [ 1.]
2669: [2669], [ 1.]
2670: [2670], [ 1.]
2671: [2671], [ 1.]
2672: [2672], [ 1.]
2673: [2673], [ 1.]
2674: [2674], [ 1.]
2675: [2675], [ 1.]
2676: [2676], [ 1.]
2677: [2677], [ 1.]
2678: [2678], [ 1.]
2679: [2679], [ 1.]
2680: [2680], [ 1.]
2681: [2681], [ 1.]
2682: [2682], [ 1.]
2683: [2683], [ 1.]
2684: [2684], [ 1.]
2685: [2685], [ 1.]
2686: [2686], [ 1.]
2687: [2687], [ 1.]
26

3893: [3893], [ 1.]
3894: [3894], [ 1.]
3895: [3895 3879 3911], [ 1.  1.  1.]
3896: [3896], [ 1.]
3897: [3881 3897 3641 3642 3898], [ 1.  1.  1.  1.  1.]
3898: [3881 3897 3641 3642 3898 3625], [ 1.  1.  1.  1.  1.  1.]
3899: [3899], [ 1.]
3900: [3900], [ 1.]
3901: [3933 3901], [ 1.  1.]
3902: [3902], [ 1.]
3903: [3903], [ 1.]
3904: [3904], [ 1.]
3905: [3905], [ 1.]
3906: [3906 3922], [ 1.  1.]
3907: [3907], [ 1.]
3908: [3908], [ 1.]
3909: [3909], [ 1.]
3910: [3910], [ 1.]
3911: [3895 3879 3911], [ 1.  1.  1.]
3912: [3912], [ 1.]
3913: [3913], [ 1.]
3914: [3914 3658], [ 1.  1.]
3915: [3915], [ 1.]
3916: [3916 3660], [ 1.  1.]
3917: [3917 3933], [ 1.  1.]
3918: [3661 3918], [ 1.  1.]
3919: [3919], [ 1.]
3920: [3920], [ 1.]
3921: [3937 3921], [ 1.  1.]
3922: [3906 3922], [ 1.  1.]
3923: [3923], [ 1.]
3924: [3924], [ 1.]
3925: [3925], [ 1.]
3926: [3926], [ 1.]
3927: [3927], [ 1.]
3928: [3928], [ 1.]
3929: [3929], [ 1.]
3930: [3930], [ 1.]
3931: [3931], [ 1.]
3932: [3932], [ 1.]
3933: [3917

In [91]:
ptr1605, ptr1606 = indptr[1605], indptr[1606]

In [96]:
xvar = chainer.Variable(x)

In [34]:
graph_arr = graph.toarray()
summed = np.sum(graph_arr, axis=-1)
graph_div = graph_arr / summed
graph_added = np.matmul(graph_arr, x)

In [24]:
selected_graph = np.matmul(graph_div, x)

In [30]:
inds_all = indices[indptr[:-1]]
data_all = data[indptr[:-1]]
data_div =

In [21]:
selected_csr = x[inds_all]

In [38]:
def csr_select_mean(csr, x):
    y = np.copy(x)
    N = x.shape[0]
    for i in range(N):
        j, k = csr.indptr[i], csr.indptr[i+1]
        cur_ind = csr.indices[j:k]
        num_ind = cur_ind.shape[0]
        y[i] = np.mean(y[cur_ind], axis=0)
    return y

def batch_csr_select_mean(csr, x):
    mb_size, N, D = x.shape
    xflat = np.reshape(x, (-1, D))
    y = np.copy(xflat)
    # csr is flattend dims
    for i in range(xflat.shape[0]):
        a,b = a,b = csr.indptr[i], csr.indptr[i+1]
        cur_ind = csr.indices[a:b]
        y[i] = np.mean(xflat[cur_ind], axis=0)
    return np.reshape(y, (x.shape))

def matmul_select_mean(arr, x):
    summed = np.sum(arr, axis=-1)
    graph_div = graph_arr / summed
    return np.matmul(graph_div, x)

def batch_matmul_select_mean(arr, x):
    mb_size, N, D = x.shape
    xflat = np.reshape(x, (-1, D))
    summed = np.sum(arr, axis=-1)
    #graph_div = graph_arr / summed
    y = np.matmul(arr / summed, xflat)
    return np.reshape(y, (mb_size, N, D))

def kdtree_select_mean(tree, x):
    y = np.copy(x)
    N = x.shape[0]
    for i in range(N):
        n_idx = tree.query_radius(x[[i]], r=0.01)
        y[i] = np.mean(x[n_idx], axis=0)
    return y

In [22]:
x10 = X[:10,:,:3]
x0 = X[0,:,:3]
rad = 0.03
x10flat = np.reshape(x10, (-1, 3))
csr_graph = radius_neighbors_graph(x0, rad, include_self=True)
graph_arr = csr_graph.toarray()
kdtree = KDTree(x10flat,leaf_size=40)

In [23]:
t0 = time.time()
cy = csr_select_mean(csr_graph, x0)
tc_fin = time.time() - t0
tstart_matmul = time.time()
my = matmul_select_mean(graph_arr, x0)
tm_fin = time.time() - tstart_matmul
print('{} {}'.format(tc_fin, tm_fin))

0.06326079368591309 0.08596968650817871


In [24]:
total_time_csr = 0
total_time_mm = 0
for i in range(100):
    x_cur = X[i]
    csr_graph = radius_neighbors_graph(x_cur, rad, include_self=True)
    csr_arr = csr_graph.toarray()
    t0 = time.time()
    cy = csr_select_mean(csr_graph, x_cur)
    total_time_csr += time.time() - t0
    tstart_matmul = time.time()
    my = matmul_select_mean(graph_arr, x_cur)
    total_time_mm += time.time() - tstart_matmul

print('{} {}'.format(total_time_csr, total_time_mm))

4.386312007904053 7.961118221282959


In [39]:
total_time_csr = 0
total_time_mm = 0
mb_size = 8
x_b = X[:8]
x_cur_flat = np.reshape(x_b, (-1, 3))
csr_graph = radius_neighbors_graph(x_cur_flat, rad, include_self=True)
#csr_arr = csr_graph.toarray()
t0 = time.time()
cy = batch_csr_select_mean(csr_graph, x_b)
total_time_csr += time.time() - t0
#tstart_matmul = time.time()
#my = matmul_select_mean(graph_arr, x_cur)
#total_time_mm += time.time() - tstart_matmul

total_time_csr
#print('{} {}'.format(total_time_csr, total_time_mm))

IndexError: index 33278 is out of bounds for axis 0 with size 32768

In [77]:
x3 = x[[3]]
print('x3:{}, {}'.format(x3, np.mean(x3,axis=0)))

x3:[[ 0.2575728   0.0016285   0.07095865]], [ 0.2575728   0.0016285   0.07095865]


In [37]:
for i in range(X.shape[0]):
    rad = 0.03
    neighbors = radius_neighbors_graph(X[i,:,:3], rad, include_self=True)
    
    

array([ 2.24294832,  1.88265181,  2.55108795])

In [41]:
time.time()

1516648670.7555723

In [65]:
for idx in range(ind.shape[0]):
    arr = ind[idx]
    if arr.size > 1:
        print('{} {}'.format(idx, arr))

5 [261 277   5]
8 [ 8 24]
12 [12 13]
13 [12 13]
20 [517  20]
21 [ 21 278]
22 [262  22]
24 [ 8 24]
28 [29 28]
29 [29 28]
54 [70 54]
60 [60 76]
61 [3644   61]
70 [70 54]
76 [60 76]
77 [318  77]
89 [345  89]
112 [3952  112]
122 [377 122]
123 [123 124 637]
124 [123 124]
130 [386 130]
140 [140 141]
141 [140 141]
146 [146 402]
154 [155 154]
155 [155 154]
162 [162 418]
178 [178 434]
183 [695 183 439 423]
184 [440 184]
212 [4052  212 4036]
261 [261 277   5]
262 [262  22]
268 [268 524]
277 [261 277   5]
278 [ 21 278]
283 [539 540 283]
292 [292 804]
293 [293 549]
295 [295 327]
299 [299 300]
300 [299 300]
301 [301 557]
309 [309 565]
310 [310 806 566 822]
312 [328 312]
313 [554 313]
314 [314 329]
315 [315 844 845]
316 [589 573 316]
317 [317 605]
318 [334 318  77]
322 [338 322]
326 [582 326]
327 [295 327]
328 [328 312]
329 [809 314 329]
333 [333 587 571]
334 [334 318]
335 [335 351]
338 [338 322]
345 [345  89]
351 [335 351]
356 [356 612]
377 [377 122]
382 [638 382]
386 [386 130]
402 [146 402]
406 [6

In [29]:
graph.indptr[20:100]

array([ 25,  27,  29,  31,  32,  34,  35,  36,  37,  39,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  67,  68,  69,  70,
        71,  72,  74,  76,  77,  78,  79,  80,  81,  82,  83,  84,  86,
        87,  88,  89,  90,  91,  93,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117], dtype=int32)